In [29]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ctr-in-advertisement/Ad_click_prediction_train (1).csv
/kaggle/input/ctr-in-advertisement/Ad_Click_prediciton_test.csv


In [30]:
train=pd.read_csv('/kaggle/input/ctr-in-advertisement/Ad_click_prediction_train (1).csv')
test=pd.read_csv('/kaggle/input/ctr-in-advertisement/Ad_Click_prediciton_test.csv')

In [31]:
train.columns

Index(['session_id', 'DateTime', 'user_id', 'product', 'campaign_id',
       'webpage_id', 'product_category_1', 'product_category_2',
       'user_group_id', 'gender', 'age_level', 'user_depth',
       'city_development_index', 'var_1', 'is_click'],
      dtype='object')

# 1.Extrapolary Data Analysis

In [32]:
train.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,session_id,DateTime,user_id,product,campaign_id,webpage_id,product_category_1,product_category_2,user_group_id,gender,age_level,user_depth,city_development_index,var_1,is_click
0,140690,2017-07-02 00:00,858557,C,359520,13787,4,NaN,10.0,Female,4.0,3.0,3.0,0,0
1,333291,2017-07-02 00:00,243253,C,105960,11085,5,NaN,8.0,Female,2.0,2.0,NaN,0,0
2,129781,2017-07-02 00:00,243253,C,359520,13787,4,NaN,8.0,Female,2.0,2.0,NaN,0,0
3,464848,2017-07-02 00:00,1097446,I,359520,13787,3,NaN,3.0,Male,3.0,3.0,2.0,1,0
4,90569,2017-07-02 00:01,663656,C,405490,60305,3,NaN,2.0,Male,2.0,3.0,2.0,1,0


>## Fixing NULL values
    
    we wll replace NAN values with -1 or unknown, rather than just drop the coulmn 

    1.ONN handles categorical data via embeddings.

    2."Unknown" can become its own learnable representation.

    3.Missing data may actually carry a pattern (e.g., unprofiled users behave differently).

In [33]:
train.isnull().sum()

session_id                     0
DateTime                       0
user_id                        0
product                        0
campaign_id                    0
webpage_id                     0
product_category_1             0
product_category_2        365854
user_group_id              18243
gender                     18243
age_level                  18243
user_depth                 18243
city_development_index    125129
var_1                          0
is_click                       0
dtype: int64

In [34]:
cols = ['user_group_id', 'gender', 'age_level', 'user_depth','city_development_index','product_category_2']
for col in cols:
    train[col] = train[col].fillna('-1').astype(str)

In [35]:
train.isnull().sum()

session_id                0
DateTime                  0
user_id                   0
product                   0
campaign_id               0
webpage_id                0
product_category_1        0
product_category_2        0
user_group_id             0
gender                    0
age_level                 0
user_depth                0
city_development_index    0
var_1                     0
is_click                  0
dtype: int64

>## Fixing target class(is_click) imbalance

    is_click, which has two classes:

    0 → No click

    1 → Click
    ------------------------------------------------------------------------

    Why should we fix this?
    

    1.CTR datasets are heavily imbalanced — most users do not click ads.

    2.If class 1 (click) is very rare (e.g., 5%), a model could predict 0 all the time and still    get 95% accuracy — but it’s useless.

    3.Imbalance can hurt training, especially for deep models like ONN.

In [36]:
train['is_click'].value_counts()


is_click
0    431960
1     31331
Name: count, dtype: int64

In [37]:
train['is_click'].value_counts(normalize=True)


is_click
0    0.932373
1    0.067627
Name: proportion, dtype: float64

given that only 6.7% of the people clicked the class is heavily imbalanced and we will fix this by using weighted loss, 

first we calculate poss weight using

pos_weight= Number of Positive Samples/Number of Negative Samples
​
 


In [38]:
import torch

# Count the labels
num_neg = (train['is_click'] == 0).sum()
num_pos = (train['is_click'] == 1).sum()


In [39]:
import torch
import torch.nn as nn

# STEP 1: Calculate pos_weight

num_neg = (train['is_click'] == 0).sum()
num_pos = (train['is_click'] == 1).sum()

pos_weight = torch.tensor([num_neg / num_pos])
print(f"pos_weight = {pos_weight}")


# STEP 2: Define a simple model


class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.fc = nn.Linear(input_dim, 1) 

    def forward(self, x):
        return self.fc(x)

model = SimpleNN(input_dim=10)

# STEP 3: Create fake inputs and targets

input_batch = torch.randn(32, 10) 
target = torch.randint(0, 2, (32,)).float() 


# STEP 4: Define the weighted loss

loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

# STEP 5: Compute the loss

logits = model(input_batch)               
logits = logits.squeeze()                 
loss = loss_fn(logits, target)            
print(f"Loss: {loss.item()}")


pos_weight = tensor([13.7870], dtype=torch.float64)
Loss: 4.720519065856934


we haven't fixed class imbalance yet, we will pass on this loss value during training to ensure that the model

That false negatives (predicting 0 when it should be 1) are expensive.

That it should balance the class decision boundary more fairly.

>## Cardinality of Categorical values

In [40]:
train.select_dtypes(include='object').nunique()


DateTime                  8610
product                     10
product_category_2          30
user_group_id               14
gender                       3
age_level                    8
user_depth                   4
city_development_index       5
dtype: int64

* since DateTime has high cardinality we will extract hours,day and week from it  

In [41]:
train['DateTime'] = pd.to_datetime(train['DateTime'])

# Extract new categorical features
train['hour'] = train['DateTime'].dt.hour.astype(str)     
train['day'] = train['DateTime'].dt.day.astype(str)        
train['weekday'] = train['DateTime'].dt.weekday.astype(str)

In [42]:
train.columns

Index(['session_id', 'DateTime', 'user_id', 'product', 'campaign_id',
       'webpage_id', 'product_category_1', 'product_category_2',
       'user_group_id', 'gender', 'age_level', 'user_depth',
       'city_development_index', 'var_1', 'is_click', 'hour', 'day',
       'weekday'],
      dtype='object')

In [43]:
train.drop(columns=['DateTime'], inplace=True)


>## Label Encoding

ONN expects every categorical feature to be represented as an integer (0, 1, 2, …), because those integers are used as indices to look up embeddings.

LabelEncoder transforms strings like 'Female', 'Male', '-1', '2.0' into integer IDs

These IDs are then used to lookup vectors from the embedding table

In [44]:
cat_cols = [
    'product', 'product_category_2', 'user_group_id',
    'gender', 'age_level', 'user_depth',
    'hour', 'day', 'weekday', 'city_development_index'
]


In [45]:
from sklearn.preprocessing import LabelEncoder

encoders = {}  

for col in cat_cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    encoders[col] = le  


In [46]:
train.head(15)

,session_id,user_id,product,campaign_id,webpage_id,product_category_1,product_category_2,user_group_id,gender,age_level,user_depth,city_development_index,var_1,is_click,hour,day,weekday
0,140690,858557,2,359520,13787,4,0,3,1,5,3,3,0,0,0,0,5
1,333291,243253,2,105960,11085,5,0,12,1,3,2,0,0,0,0,0,5
2,129781,243253,2,359520,13787,4,0,12,1,3,2,0,0,0,0,0,5
3,464848,1097446,8,359520,13787,3,0,7,2,4,3,2,1,0,0,0,5
4,90569,663656,2,405490,60305,3,0,6,2,3,3,2,1,0,0,0,5
5,151475,509591,8,359520,13787,2,0,2,2,2,3,0,0,0,0,0,5
6,17583,1091463,5,405490,60305,3,0,13,1,4,3,4,0,0,0,0,5
7,461128,469098,2,360936,13787,3,0,8,2,5,3,4,0,0,0,0,5
8,390699,611906,7,105960,11085,5,14,0,0,0,0,0,0,0,0,0,5
9,353607,418107,1,360936,13787,2,0,8,2,5,3,4,0,0,0,0,5


* All the parameters now contain numerical values which is what we wanted to achieve with the label encoding all we will do now is convert the datatype into str for each


In [47]:
for col in cat_cols:
    train[col] = train[col].astype(str)


In [48]:
train.dtypes

session_id                 int64
user_id                    int64
product                   object
campaign_id                int64
webpage_id                 int64
product_category_1         int64
product_category_2        object
user_group_id             object
gender                    object
age_level                 object
user_depth                object
city_development_index    object
var_1                      int64
is_click                   int64
hour                      object
day                       object
weekday                   object
dtype: object

# **Creating the model**

In [49]:
field_dims = [train[col].nunique() for col in cat_cols]
print("Field dimensions:", field_dims)

Field dimensions: [10, 30, 14, 3, 8, 4, 24, 6, 6, 5]


In [50]:
import torch
from sklearn.model_selection import train_test_split

# Features and labels
X = train[cat_cols].astype('int64').values
y = train['is_click'].astype('int64').values

# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.long)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.long)
y_val = torch.tensor(y_val, dtype=torch.float32)


In [51]:
import torch
import torch.nn as nn

class ONNModel(nn.Module):
    def __init__(self, field_dims, embed_dim=16, hidden_dims=[64, 32], use_outer_product=True):
        """
        Args:
            field_dims (list[int]): Number of unique values for each categorical feature
            embed_dim (int): Embedding size per feature
            hidden_dims (list[int]): Sizes of hidden MLP layers
            use_outer_product (bool): Whether to include outer-product interactions
        """
        super(ONNModel, self).__init__()

        self.num_fields = len(field_dims)
        self.embed_dim = embed_dim
        self.use_outer_product = use_outer_product

        # Operation-aware embeddings
        self.copy_embeddings = nn.ModuleList([
            nn.Embedding(field_dim, embed_dim) for field_dim in field_dims
        ])
        self.inner_product_embeddings = nn.ModuleList([
            nn.Embedding(field_dim, embed_dim) for field_dim in field_dims
        ])
        self.outer_product_embeddings = nn.ModuleList([
            nn.Embedding(field_dim, embed_dim) for field_dim in field_dims
        ]) if use_outer_product else None

        # Number of unique interaction pairs: C(n, 2)
        self.interaction_pairs = self.num_fields * (self.num_fields - 1) // 2

        # Final input size to MLP
        self.interaction_dim = self.interaction_pairs * (1 + (embed_dim if use_outer_product else 0))
        mlp_input_dim = self.num_fields * embed_dim + self.interaction_dim

        # MLP Layers
        layers = []
        for h_dim in hidden_dims:
            layers.append(nn.Linear(mlp_input_dim, h_dim))
            layers.append(nn.BatchNorm1d(h_dim))
            layers.append(nn.ReLU())
            mlp_input_dim = h_dim
        layers.append(nn.Linear(mlp_input_dim, 1))
        self.mlp = nn.Sequential(*layers)

        # Xavier Initialization
        for emb in self.copy_embeddings + self.inner_product_embeddings:
            nn.init.xavier_uniform_(emb.weight)
        if use_outer_product:
            for emb in self.outer_product_embeddings:
                nn.init.xavier_uniform_(emb.weight)

    def forward(self, x):
        """
        Forward pass of the ONN model.

        Args:
            x (LongTensor): Input tensor of shape [batch_size, num_fields], where each value is a category index.

        Returns:
            Tensor: Output tensor of shape [batch_size], containing predicted probabilities.
        """
        batch_size = x.size(0)

        # Embeddings
        copy_embeds = [emb(x[:, i]) for i, emb in enumerate(self.copy_embeddings)]  # List of [B, D]
        inner_embeds = [emb(x[:, i]) for i, emb in enumerate(self.inner_product_embeddings)]
        if self.use_outer_product:
            outer_embeds = [emb(x[:, i]) for i, emb in enumerate(self.outer_product_embeddings)]

        # Combine copy embeddings for raw features
        copy_vector = torch.cat(copy_embeds, dim=1)  # [B, num_fields * D]

        # ----- Interaction Layer: Inner and Outer Products -----
        inner_products = []
        outer_products = []

        for i in range(self.num_fields):
            for j in range(i + 1, self.num_fields):
                # Inner-product: scalar
                pij = torch.sum(inner_embeds[i] * inner_embeds[j], dim=1, keepdim=True)  # [B, 1]
                inner_products.append(pij)

                # Outer-product: element-wise vector
                if self.use_outer_product:
                    oij = outer_embeds[i] * outer_embeds[j]  # [B, D]
                    outer_products.append(oij)

        # Combine interaction outputs
        interaction_parts = [torch.cat(inner_products, dim=1)]  # [B, num_pairs]
        if self.use_outer_product:
            interaction_parts.append(torch.cat(outer_products, dim=1))  # [B, num_pairs * D]

        interaction_vector = torch.cat(interaction_parts, dim=1)  # [B, interaction_dim]

        # Final input to MLP
        model_input = torch.cat([copy_vector, interaction_vector], dim=1)  # [B, total_input_dim]

        # Output prediction
        output = self.mlp(model_input)
        return torch.sigmoid(output.squeeze(1))  # [B]


In [52]:
num_neg = (y_train == 0).sum().item()
num_pos = (y_train == 1).sum().item()
pos_weight = torch.tensor([num_neg / num_pos])
loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
model = ONNModel(field_dims, embed_dim=32, hidden_dims=[128, 64, 32])
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [53]:
batch_size = 2048
epochs = 25

for epoch in range(epochs):
    model.train()
    permutation = torch.randperm(X_train.size(0))  # ✅ define it here at the start of each epoch
    epoch_loss = 0.0

    for i in range(0, X_train.size(0), batch_size):
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_train[indices], y_train[indices]

        optimizer.zero_grad()
        logits = model(batch_x)
        loss = loss_fn(logits, batch_y)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_epoch_loss = epoch_loss / (X_train.size(0) // batch_size)
    print(f"Epoch {epoch+1}, Avg Loss: {avg_epoch_loss:.4f}")

    scheduler.step()  # if you're using a learning rate scheduler


Epoch 1, Avg Loss: 1.3483
Epoch 2, Avg Loss: 1.3282
Epoch 3, Avg Loss: 1.3168
Epoch 4, Avg Loss: 1.3088
Epoch 5, Avg Loss: 1.3028
Epoch 6, Avg Loss: 1.2974
Epoch 7, Avg Loss: 1.2946
Epoch 8, Avg Loss: 1.2922
Epoch 9, Avg Loss: 1.2899
Epoch 10, Avg Loss: 1.2877
Epoch 11, Avg Loss: 1.2845
Epoch 12, Avg Loss: 1.2829
Epoch 13, Avg Loss: 1.2815
Epoch 14, Avg Loss: 1.2803
Epoch 15, Avg Loss: 1.2789
Epoch 16, Avg Loss: 1.2767
Epoch 17, Avg Loss: 1.2758
Epoch 18, Avg Loss: 1.2751
Epoch 19, Avg Loss: 1.2743
Epoch 20, Avg Loss: 1.2735
Epoch 21, Avg Loss: 1.2723
Epoch 22, Avg Loss: 1.2718
Epoch 23, Avg Loss: 1.2714
Epoch 24, Avg Loss: 1.2710
Epoch 25, Avg Loss: 1.2706


In [54]:
from sklearn.metrics import (
    roc_auc_score, log_loss,
    precision_score, recall_score, accuracy_score
)

model.eval()
with torch.no_grad():
    val_logits = model(X_val)
    val_probs = torch.sigmoid(val_logits).numpy()  # Convert logits to probabilities
    val_preds = (val_probs >= 0.5).astype(int)      # Threshold to get binary predictions

# Ground truth labels should be numpy arrays
y_true = y_val.numpy()

# Print evaluation metrics
print("AUC:", roc_auc_score(y_true, val_probs))
print("LogLoss:", log_loss(y_true, val_probs))
print("Precision:", precision_score(y_true, val_preds))
print("Recall:", recall_score(y_true, val_preds))


AUC: 0.5517928945078235
LogLoss: 0.7812175698666252
Precision: 0.06766746889131116
Recall: 1.0
